In [1]:
import sys
import otis

otis.register()

if sys.platform == 'linux':
    port = '/dev/ttyUSB*'
elif sys.platform == 'darwin':
    port = '/dev/cu.usb*'
else:
    # port = 'COMX'
    raise EnvironmentError('Please set the correct number for the COM port')
    
from reachy import Reachy, parts

r = Reachy(
    right_arm=parts.RightArm(io=port, hand='otis')
)

otis = r.right_arm.hand

In [2]:
%matplotlib

from matplotlib import pyplot as plt

import numpy as np

# This can be installed via pip
from svgpathtools import svg2paths, wsvg

Using matplotlib backend: MacOSX


In [35]:
def svg_to_strokes(filename):
    paths, attrs = svg2paths(filename, convert_rectangles_to_paths=False)

    def pixel2mm(pix, DPI=300):
        return pix * 25.4 / DPI

    strokes = []

    for p in paths:
        pts = np.asarray([[np.real(p.point(x)), np.imag(p.point(x))] 
                          for x in np.linspace(0, 1, 30 * len(p))])
        X = pixel2mm(pts[:, 0])
        Y = pixel2mm(pts[:, 1])

        strokes.append(np.array((X, Y)).T)
        
    return strokes

In [40]:
svg_example_path = 'flower.svg'

strokes = svg_to_strokes(svg_example_path)

plt.figure()
for s in strokes:
    X, Y = s.T
    plt.plot(X, Y)
plt.gca().set_aspect('equal', adjustable='box')

In [69]:
import time

otis.compliant = False

goal, real = [], []

for s in strokes:
    otis.lift()

    x, y = s[0]
    theta_a, theta_b = otis.inverse(x, y)

    otis.goto(
        goal_positions={
            'motor_a': theta_a, 
            'motor_b': theta_b
        }, 
        duration=0.5,
        wait=True,
    )

    otis.drop()
    
    for x, y in s:
        a, b, = otis.inverse(x, y)
        otis.motor_a.goal_position = a
        otis.motor_b.goal_position = b
        time.sleep(0.025)
    
        goal.append((a, b))
        real.append((otis.motor_a.present_position, otis.motor_b.present_position))

otis.lift()

goal = np.array(goal)
real = np.array(real)

In [59]:
plt.figure()

GX, GY = otis.forward(goal[:, 0], goal[:, 1])
RX, RY = otis.forward(real[:, 0], real[:, 1])

plt.plot(GX, GY, '--')
plt.plot(RX, RY)

plt.gca().set_aspect('equal', adjustable='box')